In [16]:
import pandas as pd
from docx import Document
import os
from datetime import datetime

def generar_contratos(archivo_excel, plantilla_word, carpeta_salida="contratos_generados"):
    """
    Genera contratos automáticamente desde una plantilla Word y datos de Excel
    
    Args:
        archivo_excel (str): Ruta del archivo Excel con los datos
        plantilla_word (str): Ruta de la plantilla Word
        carpeta_salida (str): Carpeta donde guardar los contratos generados
    """
    
    # Crear carpeta de salida si no existe
    if not os.path.exists(carpeta_salida):
        os.makedirs(carpeta_salida)
    
    # Leer datos del Excel
    try:
        df = pd.read_excel(archivo_excel)
        print(f"✅ Excel leído correctamente. {len(df)} registros encontrados.")
        print(f"Columnas: {list(df.columns)}")
    except Exception as e:
        print(f"❌ Error al leer el Excel: {e}")
        return
    
    # Procesar cada fila del Excel
    contratos_generados = 0
    
    for index, row in df.iterrows():
        try:
            # Abrir la plantilla Word
            doc = Document(plantilla_word)
            
            # Diccionario con las variables y sus valores
            variables = {
                "[(CÉDULA)]": str(row["CÉDULA"]),
                "[(NOMBRE)]": str(row["NOMBRE"]),
                "[(SALARIO EN PALABRAS)]": str(row["SALARIO EN PALABRAS"]),
                "[(SALARIO EN PESOS)]": str(row["SALARIO EN PESOS"])
            }
            
            # Reemplazar variables en todos los párrafos
            for paragraph in doc.paragraphs:
                for variable, valor in variables.items():
                    if variable in paragraph.text:
                        paragraph.text = paragraph.text.replace(variable, valor)
            
            # Reemplazar variables en tablas (si las hay)
            for table in doc.tables:
                for row_table in table.rows:
                    for cell in row_table.cells:
                        for variable, valor in variables.items():
                            if variable in cell.text:
                                cell.text = cell.text.replace(variable, valor)
            
            # Reemplazar variables en headers y footers
            for section in doc.sections:
                # Header
                if section.header:
                    for paragraph in section.header.paragraphs:
                        for variable, valor in variables.items():
                            if variable in paragraph.text:
                                paragraph.text = paragraph.text.replace(variable, valor)
                
                # Footer
                if section.footer:
                    for paragraph in section.footer.paragraphs:
                        for variable, valor in variables.items():
                            if variable in paragraph.text:
                                paragraph.text = paragraph.text.replace(variable, valor)
            
            # Generar nombre del archivo
            cedula = str(row["CÉDULA"]).replace("/", "-").replace("\\", "-")
            nombre = str(row["NOMBRE"]).replace("/", "-").replace("\\", "-")
            nombre_archivo = f"Contrato_{nombre}_{cedula}.docx"
            ruta_completa = os.path.join(carpeta_salida, nombre_archivo)
            
            # Guardar el documento
            doc.save(ruta_completa)
            contratos_generados += 1
            print(f"✅ Contrato generado: {nombre_archivo}")
            
        except Exception as e:
            print(f"❌ Error procesando fila {index + 1}: {e}")
            continue
    
    print(f"\n🎉 Proceso completado!")
    print(f"📄 Contratos generados: {contratos_generados}")
    print(f"📁 Guardados en: {os.path.abspath(carpeta_salida)}")

def validar_archivos(archivo_excel, plantilla_word):
    """
    Valida que los archivos existan y tengan las columnas necesarias
    """
    # Verificar que existan los archivos
    if not os.path.exists(archivo_excel):
        print(f"❌ No se encuentra el archivo Excel: {archivo_excel}")
        return False
    
    if not os.path.exists(plantilla_word):
        print(f"❌ No se encuentra la plantilla Word: {plantilla_word}")
        return False
    
    # Verificar columnas del Excel
    try:
        df = pd.read_excel(archivo_excel)
        columnas_requeridas = ["CÉDULA", "NOMBRE", "SALARIO EN PALABRAS", "SALARIO EN PESOS"]
        columnas_faltantes = [col for col in columnas_requeridas if col not in df.columns]
        
        if columnas_faltantes:
            print(f"❌ Faltan columnas en el Excel: {columnas_faltantes}")
            print(f"Columnas encontradas: {list(df.columns)}")
            return False
        
        print("✅ Validación exitosa")
        return True
        
    except Exception as e:
        print(f"❌ Error validando Excel: {e}")
        return False

# Ejemplo de uso
if __name__ == "__main__":
    # Configuración de archivos
    ARCHIVO_EXCEL = "Prueba de Concepto.xlsx"  # Cambia por tu archivo Excel
    PLANTILLA_WORD = "Prueba de Concepto.docx"  # Cambia por tu plantilla
    CARPETA_SALIDA = "contratos_generados"
    
    print("🚀 Iniciando generación de contratos...")
    print("=" * 50)
    
    # Validar archivos antes de procesar
    if validar_archivos(ARCHIVO_EXCEL, PLANTILLA_WORD):
        generar_contratos(ARCHIVO_EXCEL, PLANTILLA_WORD, CARPETA_SALIDA)
    else:
        print("\n❌ Por favor corrige los errores antes de continuar")
        print("\n📋 Instrucciones:")
        print("1. Asegúrate de que el archivo Excel existe")
        print("2. Verifica que la plantilla Word existe")
        print("3. Confirma que el Excel tiene las columnas: CÉDULA, NOMBRE, SALARIO EN PALABRAS, SALARIO EN PESOS")

🚀 Iniciando generación de contratos...
✅ Validación exitosa
✅ Excel leído correctamente. 2 registros encontrados.
Columnas: ['CÉDULA', 'NOMBRE', 'SALARIO EN PALABRAS', 'SALARIO EN PESOS']
✅ Contrato generado: Contrato_SIMÓN_123.docx
✅ Contrato generado: Contrato_CATALINA_456.docx

🎉 Proceso completado!
📄 Contratos generados: 2
📁 Guardados en: c:\Users\USUARIO\Downloads\Proyecto Cata\contratos_generados
